In [54]:
#  Scrape data from NACCRR
#  Moved to retrieve_scraped.py

from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np


##  Scrape data from NACCRR

base_url = 'http://naccrrapps.naccrra.org/navy/directory/programs.php?program=omcc&state=CA&pagenum='

max_pages = 50

appended_data = []

for page in range(1, max_pages + 1):
    url = base_url+str(page)
    html = urlopen(url).read()
    soup = BeautifulSoup(html, "html.parser")
    td_all = soup.find('table').find_all('td')
    
    td_all_list = []
    for td in soup.find('table').find_all('td'):
        td_all_list.append(td.get_text())
        
    df = pd.DataFrame(np.array(td_all_list).reshape(int((len(td_all_list)/8)),8), # Split data into 8 columns
                  columns=['provider_name','type_of_care',
                           'address','city','state','zip','phone','email'])
    appended_data.append(df)
    
    if len(df) < 25:  # Fewer than 25 records means it's the last page, therefore terminate.
       break

appended_data = pd.concat(appended_data)


output_file = 'outputs/scraped_data.csv'
appended_data.to_csv(output_file, index=None)


print('Total rows scraped : ' + str(len(appended_data)))
print('Saved to: ' + output_file)


Total rows scraped : 1179
Saved to: outputs/scraped_data.csv


In [55]:
## Pull data from Heroku API
# Moved to retrieve_api.py

import urllib.request, json 
import pandas as pd

with urllib.request.urlopen("https://bw-interviews.herokuapp.com/data/providers") as url:
    data = json.loads(url.read().decode())
    
output_file = 'outputs/api_data.csv'

df_api = pd.DataFrame.from_records(data['providers'])
df_api.to_csv(output_file,index=None)

print("Saved to: " + output_file)


Saved to: outputs/api_data.csv


In [3]:
## Model Data
#. Merge / standardize, create source of truth.  note:  Would rather this be in SQL.  raw -> SQL -> model in SQL.

In [56]:
csv_provided = pd.read_csv('x_ca_omcc_providers.csv', names=["provider_name", "type_of_care", "address", "city", "state", "zip", "phone"])

# Would prefer to do this in SQL (matching phone format to other sources)

clean_nums = []
for phone_no in csv_provided['phone']:
    contactphone = "(%c%c%c) %c%c%c-%c%c%c%c" % tuple(map(ord,list(str(phone_no)[:10])))    
    #csv_provided['phone'] = contactphone
    clean_nums.append(contactphone)

csv_provided['phone'] = clean_nums

In [59]:
len(csv_provided)

1459

In [47]:
csv_provided.head()

,provider_name,type_of_care,address,city,state,zip,phone
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,(530) 753-8716
1,Carousel Preschool,Child Care Center,8328 Airport Blvd.,Los Angeles,CA,90045,(310) 216-6641
2,Adventure Club - Quail Glen,Child Care Center,1250 Canevari Dr,Roseville,CA,95747,(916) 772-7529
3,Adventure Club - Coyote Ridge,Child Care Center,1751 Morningstar Dr,Roseville,CA,95747,(916) 772-7271
4,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,(626) 334-1806


In [68]:
df_api.head()

,id,provider_name,phone,email,owner_name
0,prvdrs_xTVZPYTrIzsRBVJojkuHkRh6,La Rue Park Child Development Center,(530) 753-8716,snfaria@ucdavis.edu,Elizabeth Wall
1,prvdrs_9XYNilFfFjGeJjWbjP2uvLje,Carousel Preschool,(310) 216-6641,jane.uy@carouselschool.com,Shaneka Kittrell
2,prvdrs_6fwq7dwGBVctHrZWbpBuZhWJ,Adventure Club - Quail Glen,(916) 772-7529,jrutledge@roseville.ca.us,Dewayne Bunning
3,prvdrs_8YF3tyi0eeqlJm1UIeOZVx68,Adventure Club - Coyote Ridge,(916) 772-7271,vsisneroz@roseville.ca.us,Ernest Edmondson
4,prvdrs_nH2OzYh77wA1vs2aXsMqfNLx,Azusa Discovery Center,(626) 334-1806,jhdmaddox60@msn.com,William Bliss


In [69]:
appended_data.head()

,provider_name,type_of_care,address,city,state,zip,phone,email
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,(530) 753-8716,snfaria@ucdavis.edu
1,Carousel Preschool,Child Care Center,8333 Airport Blvd.,Los Angeles,CA,90045,(310) 216-6641,jane.uy@carouselschool.com
2,KinderCare Learning Center LLC,Child Care Center,2329 Vehicle Dr,Rancho Cordova,CA,95670,(916) 635-5700,301063@klcorp.com
3,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,(626) 334-1806,jhdmaddox60@msn.com
4,Little Peoples Corner Preschool/daycare,Child Care Center,3844 Walnut Drive #c,Eureka,CA,95503,(707) 445-0339,lsfloren@yahoo.com


In [58]:
#  With more time, I'd wrap this in a function and print out differences

csv_provided = csv_provided.drop_duplicates()   # 51 dupes removed
appended_data = appended_data.drop_duplicates()  # No dupes removed
df_api = df_api.drop_duplicates() # No dupes removed

In [10]:
from pandas_profiling import ProfileReport

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [51]:
ProfileReport(csv_provided, title="csv provided").to_file("data_profiles/csv_provided.html")

In [52]:
ProfileReport(appended_data, title="appended_data").to_file("data_profiles/scraped_data.html")

In [53]:
ProfileReport(df_api, title="df_api").to_file("data_profiles/api_data.html")

In [23]:
csv_provided

,provider_name,type_of_care,address,city,state,zip,phone
0,La Rue Park Child Development Center,Child Care Center,50 Atrium Way,Davis,CA,95616,(310) 514-1048
1,Carousel Preschool,Child Care Center,8328 Airport Blvd.,Los Angeles,CA,90045,(310) 514-1048
2,Adventure Club - Quail Glen,Child Care Center,1250 Canevari Dr,Roseville,CA,95747,(310) 514-1048
3,Adventure Club - Coyote Ridge,Child Care Center,1751 Morningstar Dr,Roseville,CA,95747,(310) 514-1048
4,Azusa Discovery Center,Child Care Center,155 W. Arrow Highway,Azusa,CA,91702,(310) 514-1048
...,...,...,...,...,...,...,...
1495,KCE Champions LLC @ Mason,Child Care Center,10337 San Ramon Drive,San Diego,CA,92126,(310) 514-1048
1496,Amy Lee Ulloa,Family Child Care Home,NaN,Santa Maria,CA,93455,(310) 514-1048
1497,Chapman Montessori School,Child Care Center,11832 Euclid Street,Garden Grove,CA,92840,(310) 514-1048
1498,Kid Ventures Liberty Station Preschool,Child Care Center,2865 Sims Road,San Diego,CA,92106,(310) 514-1048


In [81]:
df1 = csv_provided[['provider_name','type_of_care']]
df2 = appended_data[['provider_name','type_of_care']]

combined = pd.concat([df1, df2]).drop_duplicates()
combined2 = pd.concat([df1, df2])

,provider_name,type_of_care
0,La Rue Park Child Development Center,Child Care Center
1,Carousel Preschool,Child Care Center
2,KinderCare Learning Center LLC,Child Care Center
3,Azusa Discovery Center,Child Care Center
4,Little Peoples Corner Preschool/daycare,Child Care Center
...,...,...
24,Guidepost Montessori at Folsom,Child Care Center
0,Guidepost Montessori at Plum,Child Care Center
1,Cruz Family Child Care,Family Child Care Home
2,"Saucedo, Arlene Family Child Care",Family Child Care Home
